# 🧪 Pytest Parametrize - Complete Guide

**Concept:** Run the same test with different inputs automatically  
**Why it matters:** Write ONE test function, test MANY scenarios (DRY principle!)  
**When to use:** Testing functions with multiple input/output combinations

---

## 📚 Table of Contents

1. [The Problem: Repetitive Tests](#1-the-problem)
2. [The Solution: @pytest.mark.parametrize](#2-the-solution)
3. [Breaking Down the Syntax](#3-syntax-breakdown)
4. [Running Tests with Verbosity](#4-running-tests)
5. [Advanced Patterns](#5-advanced-patterns)
6. [Quick Reference](#6-quick-reference)

---

## 🎈 Simple Analogy: Cookie Quality Inspector

**Without parametrize:**
```
Test Cookie #1: Is it 5cm? ✓
Test Cookie #2: Is it 5cm? ✓  
Test Cookie #3: Is it 5cm? ✓
```
You write the SAME test 4 times! 😫

**With parametrize:**
```
ONE test: "Is cookie X the right size?"
Run with: Cookie #1, Cookie #2, Cookie #3, Cookie #4
```
Write ONCE, test MANY! 🎉

---

<a id="1-the-problem"></a>
## 1️⃣ The Problem: Repetitive Tests

Function we want to test:

In [ ]:
# The function to test
def add(a, b):
    return a + b

### ❌ Without Parametrize: Copy-Paste Madness

```python
def test_add_positive():
    assert add(2, 3) == 5

def test_add_zeros():
    assert add(0, 0) == 0

def test_add_negative():
    assert add(-1, 1) == 0

def test_add_large():
    assert add(100, 200) == 300
```

**Problems:**
- 4 functions doing the SAME thing
- Copy-paste code (DRY violation!)
- Change logic = edit 4 places
- Boring to write!

---

<a id="2-the-solution"></a>
## 2️⃣ The Solution: `@pytest.mark.parametrize`

**ONE function, FOUR tests!**

In [1]:
import pytest

@pytest.mark.parametrize("input_a, input_b, expected", [
    (2, 3, 5),         # Test case 1
    (0, 0, 0),         # Test case 2
    (-1, 1, 0),        # Test case 3
    (100, 200, 300),   # Test case 4
])
def test_add_scenarios(input_a, input_b, expected):
    assert add(input_a, input_b) == expected

---

<a id="3-syntax-breakdown"></a>
## 3️⃣ Breaking Down the Syntax

### Part A: The Decorator

```python
@pytest.mark.parametrize(...)
```

| Piece | Meaning |
|-------|---------|
| `@` | Apply decorator to function below |
| `pytest` | The testing library |
| `.mark` | Pytest's marking system |
| `.parametrize` | "Run this test multiple times with different values" |

### Part B: Parameter Names (First Argument)

```python
"input_a, input_b, expected"
```

A **string** with comma-separated variable names:

```
"input_a, input_b, expected"
    ↑         ↑        ↑
    │         │        └── 3rd parameter
    │         └── 2nd parameter  
    └── 1st parameter
```

⚠️ **Must match function parameters:**

```python
def test_add_scenarios(input_a, input_b, expected):
#                        ↑         ↑        ↑
#                     Must match the string!
```

### Part C: Test Data (Second Argument)

```python
[
    (2, 3, 5),        # input_a=2, input_b=3, expected=5
    (0, 0, 0),        # input_a=0, input_b=0, expected=0
    (-1, 1, 0),       # input_a=-1, input_b=1, expected=0
    (100, 200, 300),  # input_a=100, input_b=200, expected=300
]
```

**Structure:** A list of tuples. Each tuple = one test case.

```
# 3 parameter names = 3 values per tuple
(input_a, input_b, expected)
(   2,       3,       5    )
```

### Visual Flow

```
@pytest.mark.parametrize("param1, param2, param3", [...])
                              ↓       ↓       ↓
def test_something(param1, param2, param3):
                      ↓       ↓       ↓
    # Values flow into function for each test


List of tuples:          Function receives:
─────────────            ──────────────────
(2, 3, 5)       →       param1=2, param2=3, param3=5
(0, 0, 0)       →       param1=0, param2=0, param3=0
(-1, 1, 0)      →       param1=-1, param2=1, param3=0
```

---

<a id="4-running-tests"></a>
## 4️⃣ Running Tests with Verbosity

### Default output (minimal):
```bash
$ pytest test_file.py
....
4 passed in 0.01s
```

Each dot = one test. Not very informative!

### Verbose output (detailed):
```bash
$ pytest test_file.py -v
# or
$ pytest test_file.py --verbose
```

**Output:**
```
test_file.py::test_add_scenarios[2-3-5] PASSED
test_file.py::test_add_scenarios[0-0-0] PASSED
test_file.py::test_add_scenarios[-1-1-0] PASSED
test_file.py::test_add_scenarios[100-200-300] PASSED
```

Now you can see each test case! The `[2-3-5]` is the auto-generated test ID.

### What a failure looks like:

```python
@pytest.mark.parametrize("input_a, input_b, expected", [
    (2, 3, 5),
    (-1, 1, 999),  # ← Wrong expected!
])
def test_add(input_a, input_b, expected):
    assert add(input_a, input_b) == expected
```

**Output:**
```
FAILED test_file.py::test_add[-1-1-999]
    assert add(-1, 1) == 999
    assert 0 == 999  ← Shows exactly what failed!

1 passed, 1 failed
```

The `[-1-1-999]` ID tells you EXACTLY which inputs failed!

---

<a id="5-advanced-patterns"></a>
## 5️⃣ Advanced Patterns

### Pattern 1: Custom Test IDs

Make test names readable:

In [2]:
@pytest.mark.parametrize("input_a, input_b, expected", [
    pytest.param(2, 3, 5, id="positive_numbers"),
    pytest.param(0, 0, 0, id="zeros"),
    pytest.param(-1, 1, 0, id="negative_plus_positive"),
    pytest.param(100, 200, 300, id="large_numbers"),
])
def test_add_with_ids(input_a, input_b, expected):
    assert add(input_a, input_b) == expected

**Output with `-v`:**
```
test_add_with_ids[positive_numbers] PASSED
test_add_with_ids[zeros] PASSED
test_add_with_ids[negative_plus_positive] PASSED
test_add_with_ids[large_numbers] PASSED
```

Much more readable! 📖

### Pattern 2: Multiple Decorators (Cartesian Product)

Creates ALL combinations:

In [ ]:
@pytest.mark.parametrize("a", [1, 2])
@pytest.mark.parametrize("b", [10, 20])
def test_multiply_combinations(a, b):
    # This creates 4 tests: (1,10), (1,20), (2,10), (2,20)
    print(f"Testing: {a} × {b} = {a * b}")
    assert a * b == a * b

**Creates 4 tests (2 × 2):**
```
test_multiply_combinations[10-1]: a=1, b=10
test_multiply_combinations[10-2]: a=2, b=10
test_multiply_combinations[20-1]: a=1, b=20
test_multiply_combinations[20-2]: a=2, b=20
```

### Pattern 3: Skip Certain Cases

In [ ]:
@pytest.mark.parametrize("input_a, input_b, expected", [
    (2, 3, 5),
    pytest.param(0, 0, 0, marks=pytest.mark.skip(reason="Skip zeros for now")),
    (-1, 1, 0),
])
def test_add_with_skip(input_a, input_b, expected):
    assert add(input_a, input_b) == expected

### Pattern 4: Expected Failures (Known Bugs)

In [ ]:
@pytest.mark.parametrize("input_a, input_b, expected", [
    (2, 3, 5),
    pytest.param(-1, -1, -2, marks=pytest.mark.xfail(reason="Known bug #123")),
])
def test_add_with_xfail(input_a, input_b, expected):
    assert add(input_a, input_b) == expected

### Pattern 5: With Fixtures

In [ ]:
# Fixture provides reusable test setup
@pytest.fixture
def calculator():
    """Returns a calculator instance for testing."""
    class Calculator:
        def add(self, a, b):
            return a + b
    return Calculator()


@pytest.mark.parametrize("a, b, expected", [
    (2, 3, 5),
    (0, 0, 0),
])
def test_calculator_add(calculator, a, b, expected):
    #                    ↑ fixture   ↑ parametrized
    assert calculator.add(a, b) == expected

---

<a id="6-quick-reference"></a>
## 6️⃣ Quick Reference

| What You Want | How To Do It |
|---------------|-------------|
| Basic parametrize | `@pytest.mark.parametrize("a,b", [(1,2), (3,4)])` |
| Custom test names | `pytest.param(1, 2, id="my_name")` |
| Skip a case | `pytest.param(..., marks=pytest.mark.skip)` |
| Expected failure | `pytest.param(..., marks=pytest.mark.xfail)` |
| All combinations | Stack multiple `@parametrize` decorators |
| See detailed output | `pytest -v` or `pytest --verbose` |

---

## 💡 Key Takeaways

1. **One test function, many test cases** - DRY principle!
2. **Parameter names in string MUST match function arguments**
3. **Each tuple in the list = one test run**
4. **Auto-generated IDs help identify failures**
5. **Use `pytest.param()` for custom IDs and marks**
6. **Use `-v` flag to see test names**

---

## 🔨 Practice Exercise

Test the `is_even()` function with parametrize:

```python
def is_even(n):
    return n % 2 == 0
```

Test cases to include:
- Positive even (4 → True)
- Positive odd (5 → False)
- Zero (0 → True)
- Negative even (-2 → True)
- Negative odd (-3 → False)

In [ ]:
# Your solution here:
def is_even(n):
    return n % 2 == 0


@pytest.mark.parametrize("number, expected", [
    # Add your test cases!
])
def test_is_even(number, expected):
    assert is_even(number) == expected

---

## ➡️ Next Steps

**To practice:**
- Add parametrize to your regex_sum.py tests
- Use custom IDs for readable test names
- Combine with fixtures for setup/teardown

**Related topics:**
- `pytest.fixture` - Reusable test setup
- `conftest.py` - Shared fixtures across files
- `pytest.mark` - Other markers (slow, skip, xfail)